# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 48](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=48)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "TODAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "false"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=True,                # TODO change to False before push
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DBG get_records endpoint='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports'
DBG get_records qstr: is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112
DBG get_records len(page)=0 len(recs)=0
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
DBG get_exposures len(page)=0 len(recs)=0
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
DBG get_exposures len(page)=0 len(recs)=0
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_da

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-11 to 2024-11-12 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-11**

,(times in UTC),
Moon Rise,2024-11-10 17:43:00.925,
Moon Set,2024-11-11 06:41:58.146,
Moon Illumination,71%,(% illuminated)
Astronomical Twilight (morning),2024-11-11 08:10:48.514,(-18 degrees)
Astronomical Twilight (evening),2024-11-11 00:42:56.117,(-18 degrees)
Solar Midnight,2024-11-11 04:26:52.295,
Nautical Twilight (morning),2024-11-11 08:43:18.209,(-12 degrees)
Nautical Twilight (evening),2024-11-11 00:10:26.410,(-12 degrees)
Civil Twilight (morning),2024-11-11 09:14:21.799,(-6 degrees)
Civil Twilight (evening),2024-11-10 23:39:22.778,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam
Total Night,7:27:52
Total Exposure,0:00:00
Slew time(1),0:00:00
Readout time(2),0:00:11
Time loss to fault,NA
Time loss to weather,NA
Idle time,7:27:52
Number of exposures,5
Mean readout time,0.000639
Number of slews(1),0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
- Observing Operations - OBS
- TODO: better tickets using Jira API

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [OBS-651](https://rubinobs.atlassian.net/browse/OBS-651)

- [OBS-614](https://rubinobs.atlassian.net/browse/OBS-614)

- [OBS-659](https://rubinobs.atlassian.net/browse/OBS-659)

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

- Should we replace zero with empty string?
- TODO: Do not show empty tables. ie no table when "0 total exposures"
- TODO: Add link to exposures detail when A field value (e.g. observation_reason=science) is clicked on.  The detail should include all exposures counted int the total for that field value.

In [13]:
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        print(field_name.title().replace('_',' '))
        display(df.T)

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 5 total exposures**

Observation Type


,good,questionable,junk,unknown,total
bias,0,0,0,5,5


Observation Reason


,good,questionable,junk,unknown,total
bias,0,0,0,5,5


Science Program


,good,questionable,junk,total
unknown,,,,
5,0,0,0,5


**LSSTCam: 0 total exposures**

In [14]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    #  md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts,cc,f = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in []: # counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

### Experimental Exposure field Tally DISASBLED

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 5 total exposures**

**LSSTCam: 0 total exposures**

Note: *Added counts of Exposures with messages*

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-11 04:18:44.671682** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTDome telemetry returning to normal</pre>


- **2024-11-11 04:17:45.689723** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Tried move_rotator script, but failed. Can't move the rotator to enabled mode. 
The difference between the rotator and CCW positions is out of range to enable CSC.   


Alternative procedure: move CCW to rotator position through the EUI 
	Put MTRotator and MTMount in Disabled 
	Take control in the EUI 
	CCW, reset alarm, turn on, select 1, input the rotator position of 5.8 deg, move, reset alarm, turn off 
	Main axes, reset alarm


Send MTMount to Enabled 
Then  MTForceError.MTM2 fault
	 (the range is not tuned for the AOS tests)</pre>


- **2024-11-11 04:09:33.289017** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Rotator position is 5.874 and the CCW position is 3.674.</pre>


- **2024-11-11 04:06:39.542003** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTPtg wen to fault


 Error Rotator replied error for trackTarget command (1735273186) Error no:1 message:Failed: Must be in Enabled state
0</pre>


- **2024-11-11 04:06:06.989743** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTRotator wen to fault


Camera cable wrap not following closely enough: error # 3 = 2.2011443473732264 &gt; 2.2 deg</pre>


...

- **2024-11-11 14:44:02.085067** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Visual inspection with FA215 looked good, Christian and Roberto opened the FA215's valve.</pre>


- **2024-11-11 13:24:52.800785** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>LOTO on, Christian gets into the M1M3 cell to inspect FA 215</pre>


- **2024-11-11 13:10:27.057259** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>TMA parked at zenith, az=-31. locking pin in. PS off, OSS off.</pre>


- **2024-11-11 13:08:09.819671** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTPtg ENABLED, MTMount DISABLED</pre>


- **2024-11-11 12:58:37.079807** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Day crew came. Need TMA zenith and az=-31 for access to the M1M3 cell.
PS on, switched to the Parking set, David Jimenez moving TMA in EUI.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [16]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-11 12:00:00**.
- min_dayobs='2024-11-11', max_dayobs='2024-11-12'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [17]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,0,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/exposures.LSSTComCam,5,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/exposures.LSSTCam,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
narrativelog/messages,62,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-11T00%3A00%3A00&max_date_added=2024-11-12T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [18]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [19]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [0]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112

None
- Got 0 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [62]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-11T00%3A00%3A00&max_date_added=2024-11-12T00%3A00%3A00

None
- Got 62 records.  


In [20]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [21]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

## Finale

In [22]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-11 20:44:55.082635
